In [1]:
import numpy as np
import tensorflow as tf

In [2]:
from keras.datasets import mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [3]:
x_train = np.array(x_train, dtype='float32') / 255
x_test = np.array(x_test, dtype='float32') / 255

In [4]:
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)

In [5]:
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)

In [7]:
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

In [8]:
from keras import layers
from keras import models

In [9]:
model = models.Sequential()
model.add(layers.Conv2D(32,(3,3), activation='relu', input_shape = (28,28,1)))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(64,(3,3), activation='relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(64,(3,3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(64,activation = 'relu'))
model.add(layers.Dense(10, activation= 'softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten (Flatten)            (None, 576)               0         
_________________________________________________________________
dense (Dense)                (None, 64)                3

In [10]:
model.compile(optimizer = 'rmsprop',
             loss = 'categorical_crossentropy',
             metrics = ['accuracy'])

In [11]:
model.fit(x_train, y_train, epochs = 10, batch_size = 64, verbose = 1, validation_data = (x_test, y_test))

Epoch 1/10
938/938 [==============================] - 70s 73ms/step - loss: 0.1648 - accuracy: 0.9483 - val_loss: 0.0487 - val_accuracy: 0.9836
Epoch 2/10
938/938 [==============================] - 66s 70ms/step - loss: 0.0461 - accuracy: 0.9862 - val_loss: 0.0297 - val_accuracy: 0.9907
Epoch 3/10
938/938 [==============================] - 70s 75ms/step - loss: 0.0319 - accuracy: 0.9904 - val_loss: 0.0312 - val_accuracy: 0.9901
Epoch 4/10
938/938 [==============================] - 71s 76ms/step - loss: 0.0239 - accuracy: 0.9930 - val_loss: 0.0268 - val_accuracy: 0.9910
Epoch 5/10
938/938 [==============================] - 73s 78ms/step - loss: 0.0200 - accuracy: 0.9942 - val_loss: 0.0296 - val_accuracy: 0.9911
Epoch 6/10
938/938 [==============================] - 68s 73ms/step - loss: 0.0161 - accuracy: 0.9952 - val_loss: 0.0354 - val_accuracy: 0.9897
Epoch 7/10
938/938 [==============================] - 76s 81ms/step - loss: 0.0133 - accuracy: 0.9960 - val_loss: 0.0309 - val_accuracy:

In [13]:
y_pred = model.predict(x_test)

In [14]:
pred = np.array([np.argmax(pr) for pr in y_pred])
true = np.array([np.argmax(tst) for tst in y_test])

In [15]:
y_test_all = [0] * 10
y_test_true = [0] * 10
for i in range(y_test.shape[0]):
    y_test_all[true[i]] += 1
    if true[i] == pred[i]:
        y_test_true[true[i]] += 1

In [16]:
print("Test: ")
for i in range(10):
    print("acc ", i, " - ", format((y_test_true[i] * 100) / y_test_all[i], '.4f') )

Test: 
acc  0  -  99.6939
acc  1  -  99.9119
acc  2  -  99.5155
acc  3  -  99.8020
acc  4  -  99.2872
acc  5  -  98.4305
acc  6  -  98.6430
acc  7  -  99.0272
acc  8  -  99.2813
acc  9  -  99.0089


In [17]:
from sklearn.metrics import classification_report

In [18]:
target_names = ['Класс ' + str(cls) for cls in range(10)]

In [19]:
print(classification_report(true, pred,
          target_names = target_names))

              precision    recall  f1-score   support

     Класс 0       0.99      1.00      0.99       980
     Класс 1       0.99      1.00      1.00      1135
     Класс 2       0.99      1.00      0.99      1032
     Класс 3       0.99      1.00      0.99      1010
     Класс 4       1.00      0.99      0.99       982
     Класс 5       0.99      0.98      0.99       892
     Класс 6       1.00      0.99      0.99       958
     Класс 7       0.99      0.99      0.99      1028
     Класс 8       0.99      0.99      0.99       974
     Класс 9       0.99      0.99      0.99      1009

    accuracy                           0.99     10000
   macro avg       0.99      0.99      0.99     10000
weighted avg       0.99      0.99      0.99     10000



$ PRECISION = \frac{TP}{TP + FP} \quad RECALL = \frac{TP}{TP + FN} \quad f1 = \frac{2 × precision × recall}{precision + recall} $

In [33]:
print('Класс precision  recall  f1-score')
for cls in range(10):
    TP_plus_FP = np.sum(pred == cls)
    TP_plus_FN = np.sum(true == cls)
    TP = np.sum(pred == true, where = true == cls)
    P = TP / TP_plus_FP
    R = TP / TP_plus_FN
    F1 = (2 * P * R) / (P + R)
    print(cls, '\t', round(P, 2), '\t', round(R, 2), '\t', round(F1, 2), '\t', y_test_all[cls])

Класс precision  recall  f1-score
0 	 0.99 	 1.0 	 0.99 	 980
1 	 0.99 	 1.0 	 1.0 	 1135
2 	 0.99 	 1.0 	 0.99 	 1032
3 	 0.99 	 1.0 	 0.99 	 1010
4 	 1.0 	 0.99 	 0.99 	 982
5 	 0.99 	 0.98 	 0.99 	 892
6 	 1.0 	 0.99 	 0.99 	 958
7 	 0.99 	 0.99 	 0.99 	 1028
8 	 0.99 	 0.99 	 0.99 	 974
9 	 0.99 	 0.99 	 0.99 	 1009
